In [71]:
import numpy as np
from scipy.stats import poisson, uniform, expon
import math

## record your uni here

uni = ''

## Problem 1 (25 points)

Let $$\theta = \int_0^1 \int_0^y \exp(-x^2) x^4 dx dy$$


Use the monte carlo method to estimate $\theta$.

In [8]:
np.random.seed(42)
n = 10_000

U = uniform.rvs(size=(n,2))

f = np.exp(-U[:,0]**2) * (U[:,0] ** 4)
loc = U[:,0] > U[:,1]
f[loc] = 0

theta = np.mean(f)
print(theta)

0.020152448865945565


In [19]:
import random

def monte_carlo_estimation(num_samples):
    total = 0
    for _ in range(num_samples):
        x = random.uniform(0, 1)
        y = random.uniform(0, x)  # y ranges from 0 to x
        total += (math.exp(-x**2) * x**4)
    return total / num_samples

num_samples = 10000  # You can adjust the number of samples for accuracy
estimated_theta = monte_carlo_estimation(num_samples)
print("Estimated theta using Monte Carlo method:", estimated_theta)


Estimated theta using Monte Carlo method: 0.09890704428403443


## Problem 2
## Problem Statement

Suppose the number of accidents on a highway on a given day follows a Poisson with distribution with parameter $\lambda$, depending on weather conditions.

- 85\% of days have good conditions
- 10\% of days have rain
- 5\% of days have snow


If a day has good conditions, the number of accidents is expected to be 0.8. If a day has rain, the number of accident is expected to be 2, and if it is snowing, we expect to have 4 accidents.



## Part I (25 points)

Write a function accidents(n) that generates a sample of size from the distribution of daily accidents.

Using $10,000$ simulations, calculate the probability that we have (strictly) greater than $4$ accidents in a day, and a $95\%$ confidence for this probability.

For the confidence interval, feel free to use the normal critical value $z_{.025}$ instead of $t_{.025, 9,999}$

In [55]:
np.random.seed(42)
n = 10000
def accidents(n):
    U = uniform.rvs(size=n)
    X = np.zeros(n)
    for i in range(n):
        if U[i] <= 0.85:
            lam = 0.8
        elif U[i] <= 0.95:
            lam = 2
        else:
            lam = 4
        X[i] = poisson.rvs(lam)
    return X

In [50]:
X = accidents(n)
X

array([1., 2., 0., ..., 1., 1., 0.])

In [54]:
Y = X > 4
mean = np.mean(Y)
sd = np.std(Y)
ci_low = mean - 1.96 * sd / np.sqrt(n)
ci_up = mean + 1.96 * sd / np.sqrt(n)

print(mean, ci_low, ci_up)

0.0242 0.021188074944889897 0.027211925055110102


## Part II (10 points)

If there are 3 accidents in a day, what is the probability it was snowing?

Hint: modify the function accidents to also return the type of day it was.

In [67]:
np.random.seed(42)

def accidents(n):
    U = uniform.rvs(size=n)
    X = np.zeros(n)
    lam = np.zeros(n)
    for i in range(n):
        if U[i] <= 0.85:
            lam[i] = 0.8
        elif U[i] <= 0.95:
            lam[i] = 2
        else:
            lam[i]= 4
        X[i] = poisson.rvs(lam[i])
    return X, lam

X, lam = accidents(n)
loc = X == 3
p_3 = np.mean(lam[loc] == 4)
print(p_3)

0.1348122866894198


## Problem 3 (25 points)

Suppose we want to sample from the distribution with the cdf:

$$F(x) = 1 - \exp(-\sqrt{x})$$

Write a function inverse_transform() that samples from this distribution, and calculate the expected value of this distribution using $10,000$ samples.

In [77]:
np.random.seed(42)
n = 10_000

def inverse_transform(n):
    U = uniform.rvs(size=n)
    return np.log(1-U)**2

In [78]:
np.mean(inverse_transform(n))

1.904873792779115

In [79]:
np.random.seed(42)
n = 10_000

## 1 - \exp(-sqrt(x)) = u
## -sqrt(x) = log(1 - u)
## sqrt(x) = -log(1-u)
## x = log(1-u)**2


def inverse_transform():
    U = uniform.rvs()
    return np.log(1-U)**2

X = [inverse_transform() for i in range(0, n)]
print(np.mean(X))

1.904873792779115


## Problem 4

## Part I (10 points)

Let $f(x) = 0.5\sin(x) 1_{\{0< x < \pi\}}$

Implement the rejection sampling algorithm for this distribution.

In [80]:
# we use Unif(0,pi) as our proposal distribution
# c = max f(x) / g(x) = max 0.5 * sin(x) / (1 / pi) = 0.5 * pi
# f(x) / (c g(x)) = sin(x)

def rej_sampling():
    U1 = np.pi * uniform.rvs()
    while True:
        U2 = uniform.rvs()
        if U2 < np.sin(U1):
            return U1
        U1 = np.pi * uniform.rvs()

## Part II (10 points)

Using $n=10,000$ simulations, calculate a 95% confidence interval for $E[X^3]$ where $X$ has the density listed above.

In [81]:
n = 10_000
X3 = [rej_sampling()**3 for i in range(n)]
EX3 = np.mean(X3)
sig = np.std(X3)

lower = EX3 - 1.96 * sig / np.sqrt(n)
upper = EX3 + 1.96 * sig / np.sqrt(n)
print(lower, upper)

5.939030211433355 6.183289625575499
